### **RunnableParallel**

데이터를 병렬로 조작하는 Runnable 

- 하나의 `Runnable` 의 출력을 다음 `Runnable` 의 입력 형식에 맞게 조작하는 데 유용하게 사용

In [1]:
from operator import itemgetter 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [ ]:
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
)
# 체인을 구성합니다.
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

chain.invoke({"question": "What is Teddy's occupation?", "language": "Korean"})

'테디의 직업은 AI 엔지니어입니다.'

### **병렬처리 단계적 이해**

- `RunnableParallel`을 사용하여 여러 Runnable을 병렬적으로 실행

In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'o3-mini') 

# 수도 질문 Chain
capital_chain = (
    PromptTemplate.from_template("{country} 의 수도는 어디입니까?, {city}와의 거리는 얼마입니까")
    | llm
    | StrOutputParser()
)

# 면적 질문 Chain
area_chain = (
    PromptTemplate.from_template("{country} 의 면적은 얼마입니까?, {city}와의 크기차이는 얼마입니까 면적의 단위는 제곱킬로미터로 계산해주세요")
    | llm
    | StrOutputParser()
)

# capital_chain, area_chain을 RunnableParallel로 묶음
map_chain = RunnableParallel(capital=capital_chain, area=area_chain)

map_chain.invoke({'country': '미국', 'city': '서울'})

{'capital': '미국의 수도는 워싱턴 D.C.입니다. \n\n서울과 워싱턴 D.C. 사이의 거리는 측정 방법에 따라 다를 수 있는데, 보통 직선(대권) 거리는 약 11,000km 정도로 알려져 있습니다. 실제 항공편으로 이동할 경우 경로, 경유지, 기상 여건 등에 따라 거리가 달라질 수 있으니 참고용으로 이해하시면 좋겠습니다.',
 'area': '아래는 대략적인 수치입니다(출처나 계산 방식에 따라 약간의 차이가 있을 수 있습니다).\n\n1. 미국의 전체 면적(육지 및 수역 포함)은 약 9,833,520 제곱킬로미터입니다.  \n2. 서울의 면적은 약 605 제곱킬로미터 정도입니다.\n\n따라서 두 지역의 면적 차이는  \n\u2003\u20039,833,520 km² – 605 km² ≒ 9,832,915 km²  \n이며, 미국의 면적은 서울의 면적보다 약 16,250배 정도 큽니다.'}